In [0]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

In [0]:
file_path = 'retail_data.bronze.silver_online_retail'

In [0]:
sliver_df = spark.read.table(file_path) 

display(sliver_df)

In [0]:
from pyspark.sql.functions import col
sliver_df = sliver_df \
    .withColumn('CustomerID',col('CustomerID').cast('int'))

In [0]:
from pyspark.sql.functions import when, datediff, lit,  max as spark_max,countDistinct

gold_customer_df = (
    sliver_df
    .groupBy("CustomerID")
    .agg(
        spark_max("total_amount").alias("total_spend"),
        countDistinct("InvoiceNo").alias("total_orders"),
        spark_max("InvoiceDate").alias("last_purchase_date")
    )
)

display(gold_customer_df)

In [0]:
gold_customer_df.write.mode('overwrite').format('delta').saveAsTable('retail_data.gold.gold_online_retail')